In [1]:
import paths
import sgf

In [2]:
import numpy as np

In [3]:
import keras
import tensorflow as tf
from keras import backend as K
from keras.layers import Convolution2D, ZeroPadding2D, BatchNormalization, Dense

Using TensorFlow backend.


In [4]:
def limit_mem():
    cfg = K.tf.ConfigProto()
    cfg.gpu_options.allow_growth = True
    K.set_session(K.tf.InteractiveSession(config=cfg))
limit_mem()

In [5]:
library = sgf.Library()

In [6]:
names = list(library)

n = 100
tr, val, te = int(n*0.7), int(n*0.2), int(n*0.1)

random_boards = np.random.choice(names, size=tr+val+te, )

train, valid, test = random_boards[:tr], random_boards[tr:tr+val], random_boards[tr+val:tr+val+te]

In [7]:
def game_generator(game_names, batch_size):
    Xgames = []
    Ygames = []
    while True:

        name = np.random.choice(game_names)
        move_idx = np.random.choice(len(library[name]['graymoves']))
        
        Xgames.append(library[name]['gray'][move_idx,:,:])
        Ygames.append(library[name]['graymoves'][move_idx,:,:])
                     
        if len(Xgames) >= batch_size:
            Xgames = np.array(Xgames).reshape(-1,19,19,1)
            Ygames = np.array(Ygames).reshape(-1,19*19)
            
            yield Xgames, Ygames
            
            Xgames = []
            Ygames = []

In [8]:
n = len(library)
tr, val = int(n*0.7), int(n*0.2)
te = n - tr - val

random_boards = np.random.choice(names, size=n,)

train, valid, test = random_boards[:tr], random_boards[tr:tr+val], random_boards[tr+val:tr+val+te]

In [9]:
with open('trainvalidtest.csv', 'w') as datasetup:
    for number in (tr,val,te):
        datasetup.write(str(number)+'\n')
    for game in random_boards:
        datasetup.write(game+'\n')

In [10]:
for X,Y in game_generator(test, 10):
    print(X.shape, Y.shape)
    break

(10, 19, 19, 1) (10, 361)


In [11]:
def convblock(connectinglayer, conv_filters = 32, act_function='selu', batchnormalize=True):
    edge_padding = (1,1)
    
    zero = ZeroPadding2D(edge_padding)(connectinglayer)
    conv = Convolution2D(filters=conv_filters, 
                         kernel_size=(3,3),
                         activation=act_function,
                        )(zero)
    if batchnormalize:
        lastlayer = BatchNormalization()(conv)
    else:
        lastlayer = conv
    
    return lastlayer

In [12]:
def denseblock(connectinglayer, output_size = 4096, act_function='selu', batchnormalize=True):
    dense = Dense(output_size, activation=act_function)(connectinglayer)
    if batchnormalize:
        lastlayer = BatchNormalization()(dense)
    else:
        lastlayer = dense
    
    return lastlayer

In [13]:
def make_model():
    gray_board = keras.layers.Input(shape=(19,19,1))

    current_layer = gray_board

    for _ in range(5):
        current_layer = convblock(current_layer)

    current_layer = keras.layers.Flatten()(current_layer)

    for _ in range(2):
        current_layer = denseblock(current_layer)

    move_pred = Dense(19*19, activation='sigmoid')(current_layer)


    model = keras.models.Model(gray_board, move_pred)

    model.compile(optimizer=keras.optimizers.SGD(lr=0.01), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    
    return model


X1 = library[train[0]]['gray'][()][:-1,:,:].reshape(-1,19,19,1)
y1 = library[train[0]]['graymoves'][()].reshape(-1,19*19)

X1.shape, y1.shape

model.fit(X1, y1);

model.predict(library[valid[0]]['gray'][()][100,:,:].reshape(1,19,19,1))

for game in train:
    X1 = library[game]['gray'][()][:-1,:,:].reshape(-1,19,19,1)
    y1 = library[game]['graymoves'][()].reshape(-1,19*19)
    model.fit(X1, y1, epochs=10)

In [ ]:
model = make_model()

In [ ]:
epochs = 10
batch_size = 64
training_steps = 128 * (tr // batch_size)
valid_size = min(batch_size, val)
valid_steps = 128 * (val // valid_size)

In [15]:
model.fit_generator(generator=game_generator(train, batch_size=batch_size), 
                    epochs=epochs, 
                    steps_per_epoch=training_steps, 
                    validation_data=game_generator(valid, batch_size=valid_size), 
                    validation_steps=valid_steps,
                    callbacks=[keras.callbacks.ModelCheckpoint(filepath='model_{epoch:02d}_{val_loss:.2f}.hdf5')]
                   );

Epoch 1/10
    3/71936 [..............................] - ETA: 26532s - loss: 1.5692 - categorical_accuracy: 0.5990

C:\Anaconda3\envs\gofamiliar\lib\site-packages\keras\callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.164075). Check your callbacks.
  % delta_t_median)


    4/71936 [..............................] - ETA: 24985s - loss: 1.4197 - categorical_accuracy: 0.6250

C:\Anaconda3\envs\gofamiliar\lib\site-packages\keras\callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.125001). Check your callbacks.
  % delta_t_median)


71936/71936 [==============================] - 11583s - loss: 1.4133 - categorical_accuracy: 0.6121 - val_loss: 3.1760 - val_categorical_accuracy: 0.4062
Epoch 2/10
71936/71936 [==============================] - 11733s - loss: 1.3115 - categorical_accuracy: 0.6351 - val_loss: 3.2230 - val_categorical_accuracy: 0.4169
Epoch 3/10
71936/71936 [==============================] - 11783s - loss: 1.2249 - categorical_accuracy: 0.6551 - val_loss: 3.2387 - val_categorical_accuracy: 0.4269
Epoch 4/10
71936/71936 [==============================] - 12400s - loss: 1.1467 - categorical_accuracy: 0.6734 - val_loss: 3.2578 - val_categorical_accuracy: 0.4362
Epoch 5/10
71936/71936 [==============================] - 12593s - loss: 1.0738 - categorical_accuracy: 0.6907 - val_loss: 3.3108 - val_categorical_accuracy: 0.4400
Epoch 6/10
71936/71936 [==============================] - 12259s - loss: 1.0101 - categorical_accuracy: 0.7059 - val_loss: 3.3267 - val_categorical_accuracy: 0.4509
Epoch 7/10
71936/7193